Import the necessary libraries

In [1]:
!pip install transformers
!pip install ipywidgets
!pip install pytorch-lightning==1.5.10
!pip install nvidia-ml-py3
!pip install neptune-client
!pip install lightning-bolts
!pip install torchmetrics


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached pytorch_lightning-1.5.10-py3-none-any.whl (527 kB)
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.9.5
    Uninstalling pytorch-lightning-1.9.5:
      Successfully uninstalled pytorch-lightning-1.9.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lightning-bolts 0.7.0 requires pytorch-lightning<2.0.0,>1.7.0, but you have pytorch-lightning 1.5.10 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pytorch_lightning-1.9.5-py3-none-any.whl (829 kB)



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.5.10
    Uninstalling pytorch-lightning-1.5.10:
      Successfully uninstalled pytorch-lightning-1.5.10



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model, Wav2Vec2Config
import torch
from transformers.models.wav2vec2.modeling_wav2vec2 import Wav2Vec2FeatureEncoder, Wav2Vec2NoLayerNormConvLayer, Wav2Vec2LayerNormConvLayer
from torch import nn
from transformers.activations import ACT2FN
import ipywidgets
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import torchaudio
import torchtext
import pytorch_lightning as pl
import nvidia_smi
from pytorch_lightning.loggers.neptune import NeptuneLogger
from pytorch_lightning.loggers import NeptuneLogger
from IPython.display import display, HTML
from dataclasses import dataclass, field
from torch.utils.data import DataLoader
from typing import Any, Dict, List, Optional, Union
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR
from torchmetrics import Accuracy
from torchmetrics import F1Score
import torch.nn.functional as F
import numpy as np
import contextlib

d:\Programas\Ananconda3\lib\site-packages\pkg_resources\__init__.py:116: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(
d:\Programas\Ananconda3\lib\site-packages\pkg_resources\__init__.py:116: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(
d:\Programas\Ananconda3\lib\site-packages\neptune\common\warnings.py:62: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  warnings.warn(
d:\Programas\Ananconda3\lib\site-packages\pl_bolts\models\self_supervised\amdim\amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The AP

Verify if the GPU is working

In [3]:
!nvidia-smi

Sun Oct 29 11:42:02 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.79                 Driver Version: 531.79       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650       WDDM | 00000000:01:00.0  On |                  N/A |
| N/A   58C    P8                4W /  N/A|    747MiB /  4096MiB |      8%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
print(f"Pytorch Lightning Version: {pl.__version__}")
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
print(f"Device name: {nvidia_smi.nvmlDeviceGetName(handle)}")

Pytorch Lightning Version: 1.9.5
Device name: NVIDIA GeForce GTX 1650


Define hyperparameters

In [6]:
version = "wav2vec2-sound_sismic_train" #@param {type: "string"}
lr = 1e-5#@param {type: "number"}
w_decay = 0#@param {type: "number"}
bs = 16#@param {type: "integer"}
accum_grads = 4#@param {type: "integer"}
patience = 30#@param {type: "integer"}
max_epochs = 300#@param {type: "integer"}
warmup_steps = 1000#@param {type: "integer"}
hold_epochs = 20#@param {type: "integer"}
pretrained = "facebook/wav2vec2-base-960h"#@param {type: "string"}
wav2vec2_processor = "facebook/wav2vec2-base-960h"#@param {type: "string"}
freeze_finetune_updates = 0#@param {type: "integer"}
warmup_epochs = 40#@param {type: "integer"}
apply_mask=True#@param {type: "boolean"}
mask_time_length= 10#@param {type: "integer"}, era 1

# Define hyperparameters
hparams = {"version": version,
          "lr": lr,
          "w_decay": w_decay,
          "bs": bs,
          "patience": patience,
          "hold_epochs":hold_epochs,
          "accum_grads": accum_grads,
          "pretrained":pretrained,
          "wav2vec2_processor": wav2vec2_processor,
          "freeze_finetune_updates":freeze_finetune_updates,
          "warmup_epochs":warmup_epochs,
          "apply_mask":apply_mask,
          "mask_time_length":mask_time_length,
          "max_epochs": max_epochs}
hparams

{'version': 'wav2vec2-sound_sismic_train',
 'lr': 1e-05,
 'w_decay': 0,
 'bs': 16,
 'patience': 30,
 'hold_epochs': 20,
 'accum_grads': 4,
 'pretrained': 'facebook/wav2vec2-base-960h',
 'wav2vec2_processor': 'facebook/wav2vec2-base-960h',
 'freeze_finetune_updates': 0,
 'warmup_epochs': 40,
 'apply_mask': True,
 'mask_time_length': 10,
 'max_epochs': 300}

Use the original processor from Wav2Vec2

In [7]:
processor = Wav2Vec2Processor.from_pretrained(hparams["wav2vec2_processor"], return_attention_mask=True)

d:\Programas\Ananconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\paulq\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
print(processor)

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='facebook/wav2vec2-base-960h', vocab_size=32, model_max_length=9223372036854775807, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True)


Generalize the model to accept n channels instead of just 1

In [10]:
class Wav2Vec2GroupNormConvLayer(nn.Module):
    def __init__(self, config, num_input_channels=1, layer_id=0):
        super().__init__()
        self.num_input_channels = num_input_channels
        self.in_conv_dim = config.conv_dim[layer_id - 1] if layer_id > 0 else self.num_input_channels
        self.out_conv_dim = config.conv_dim[layer_id]

        self.conv = nn.Conv1d(
            self.in_conv_dim,
            self.out_conv_dim,
            kernel_size=config.conv_kernel[layer_id],
            stride=config.conv_stride[layer_id],
            bias=config.conv_bias,
        )
        self.activation = ACT2FN[config.feat_extract_activation]

        self.layer_norm = nn.GroupNorm(num_groups=self.out_conv_dim, num_channels=self.out_conv_dim, affine=True)

    def forward(self, hidden_states):
        hidden_states = self.conv(hidden_states)
        hidden_states = self.layer_norm(hidden_states)
        hidden_states = self.activation(hidden_states)
        return hidden_states

class Wav2Vec2_ChannelFeatureEncoder(nn.Module):
    """Construct the features from raw audio waveform"""

    def __init__(self, config, num_input_channels=1):
        super().__init__()
        
        self.num_input_channels = num_input_channels
        
        if config.feat_extract_norm == "group":
            conv_layers = [Wav2Vec2GroupNormConvLayer(config, num_input_channels= self.num_input_channels,layer_id=0)] + [
                Wav2Vec2NoLayerNormConvLayer(config, layer_id=i + 1) for i in range(config.num_feat_extract_layers - 1)
            ]
        elif config.feat_extract_norm == "layer":
            conv_layers = [
                Wav2Vec2LayerNormConvLayer(config, layer_id=i) for i in range(config.num_feat_extract_layers)
            ]
        else:
            raise ValueError(
                f"`config.feat_extract_norm` is {config.feat_extract_norm}, but has to be one of ['group', 'layer']"
            )
        self.conv_layers = nn.ModuleList(conv_layers)
        self.gradient_checkpointing = False
        self._requires_grad = True

    def _freeze_parameters(self):
        for param in self.parameters():
            param.requires_grad = False
        self._requires_grad = False

    def forward(self, input_values):
        hidden_states = input_values[:] # mudou para que receba todos os canais (4)
        #print("hidden_states", hidden_states.shape)

        # make sure hidden_states require grad for gradient_checkpointing
        if self._requires_grad and self.training:
            hidden_states.requires_grad = True

        for conv_layer in self.conv_layers:
            if self._requires_grad and self.gradient_checkpointing and self.training:

                def create_custom_forward(module):
                    def custom_forward(*inputs):
                        return module(*inputs)

                    return custom_forward

                hidden_states = torch.utils.checkpoint.checkpoint(
                    create_custom_forward(conv_layer),
                    hidden_states,
                )
            else:
                hidden_states = conv_layer(hidden_states)

        return hidden_states

# Crio o novo modelo que herda os processos de Wav2Vec2, mas usa o extrator de features baseado em 4 canais
class Wav2Vec2_ChannelModel(Wav2Vec2Model):
    def __init__(self, config: Wav2Vec2Config, num_input_channels=1):
        super().__init__(config)

        #del self.feature_extractor
        self.feature_extractor = Wav2Vec2_ChannelFeatureEncoder(config, num_input_channels=num_input_channels)

In [11]:
model = Wav2Vec2_ChannelModel.from_pretrained("facebook/wav2vec2-base-960h",
                                                 conv_dim = (512, 512, 512,512,512,512),
                                                 conv_stride = (5, 2, 2,2,2,2),
                                                 conv_kernel = (10, 3, 3,3,3,2),
                                                 num_feat_extract_layers = 6,
                                                 num_input_channels = 1,
                                                 ignore_mismatched_sizes=True)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2_ChannelModel: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'lm_head.bias', 'lm_head.weight', 'wav2vec2.feature_extractor.conv_layers.6.conv.weight', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2_ChannelModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2_ChannelModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2_ChannelModel were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv

In [12]:
print(model)

Wav2Vec2_ChannelModel(
  (feature_extractor): Wav2Vec2_ChannelFeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmb

DO THE TRAIN-TEST SPLIT

In [17]:
# from sklearn.model_selection import train_test_split
# import os
# import shutil
# import librosa

# #Define la ruta de la carpeta que contiene los archivos de audio
# input_folder = 'E:/TESIS/DataSet/AudioSismig/'

# #Lista todos los archivos de la carpeta
# files = [os.path.join(input_folder, file) for file in os.listdir(input_folder)]

# #Define las proporciones para train, test y validation sets
# train_ratio = 0.7
# test_ratio = 0.15
# validation_ratio = 0.15

# ##Divide los datos en train, test y validation sets
# train_files, temp_files = train_test_split(files, test_size=1 - train_ratio)
# test_files, validation_files = train_test_split(temp_files, test_size=validation_ratio / (test_ratio + validation_ratio))

# #Define las carpetas de salida
# output_folder = './data/'
# os.makedirs(os.path.join(output_folder, 'train'), exist_ok=True)
# os.makedirs(os.path.join(output_folder, 'test'), exist_ok=True)
# os.makedirs(os.path.join(output_folder, 'validation'), exist_ok=True)


# #Copia los archivos a las carpetas correspondientes
# for file in train_files:
#     try:
#         audio, _ = librosa.load(file, sr=None)
#         shutil.copy(file, os.path.join(output_folder, 'train'))
#     except Exception as e:
#         print(e)
#         print(file)
# for file in test_files:
#     try:
#         audio, _ = librosa.load(file, sr=None)
#         shutil.copy(file, os.path.join(output_folder, 'test'))
#     except Exception as e:
#         print(e)
#         print(file)
    
# for file in validation_files:
#     try:
#         audio, _ = librosa.load(file, sr=None)
#         shutil.copy(file, os.path.join(output_folder, 'validation'))
#     except Exception as e:
#         print(e)
#         print(file)

# print("Data split completed.")


<ipython-input-17-79cb622c246c>:31: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file, sr=None)
d:\Programas\Ananconda3\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



E:/TESIS/DataSet/AudioSismig/63e65c8adae78edb6ca641a368ae9cc1_BHN_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/424df27a90cd2e79da4c8db8b0c5cf12_BDF_BNAS_VLP_TYPE2.wav

E:/TESIS/DataSet/AudioSismig/838b0472787cfcdbfbbf3b8f3e035e2d_BHN_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/f01547b1946b1922e4f6876fd40ce9da_BDF_BNAS_VLP_TYPE1.wav

E:/TESIS/DataSet/AudioSismig/eadce4cf2ff973534d62f3b4940eb4a4_BDF_BNAS_VLP_TYPE1.wav

E:/TESIS/DataSet/AudioSismig/c8e1c4f29035a394492ea4d603eafea3_BHN_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/406ab147a90b09e29f51852584f47db6_BHZ_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/508064df8e817ad664a012e558a2782a_BHZ_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/a74fdea1bbb9e5ef0091c803de836c95_BHN_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/d415433dd8f7fea18a2f5ecf69a63a2e_BHN_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/beb17c037f2ea59f6d849756a8b69344_BHE_BNAS_VLP_TYPE1.wav

E:/TESIS/DataSet/AudioSismig/f91ec4d82f9d8ba8b7ead9a046c4a1c9_BHE_BNAS_VLP_TYPE2.wav

E:/TESIS/DataSet

<ipython-input-17-79cb622c246c>:38: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file, sr=None)



E:/TESIS/DataSet/AudioSismig/c0463d86042564551956314e85c70ec6_BDF_BNAS_VLP_TYPE1.wav

E:/TESIS/DataSet/AudioSismig/794ef03d6a56e6fe07e1528e2387bd06_BDF_BNAS_VLP_TYPE1.wav

E:/TESIS/DataSet/AudioSismig/0ab4a8099b356b53644cdbbe2957121e_BHE_BNAS_VLP_TYPE2.wav

E:/TESIS/DataSet/AudioSismig/346483c520ba9e3c264b3686adef21db_BHZ_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/0002a32d9f7576599e5f41f7333c0098_BHE_BNAS_VLP_TYPE1.wav

E:/TESIS/DataSet/AudioSismig/b95f7f47654f06a812e14b21fca224b5_BDF_BNAS_VLP_TYPE1.wav

E:/TESIS/DataSet/AudioSismig/2e583f7bb7e30c9385f8793685f25cb6_BDF_BNAS_VLP_TYPE1.wav

E:/TESIS/DataSet/AudioSismig/51faf8e5cab4e392a9728be06576d63b_BHN_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/920d32179c8fc49eaffc5611022999a1_BDF_BNAS_VLP_TYPE2.wav

E:/TESIS/DataSet/AudioSismig/14f881550a7a283acf1118cc2844f55f_BHN_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/5f86c4bbfd4e783ff0a3a1d51e8be597_BHZ_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/4aecb0ba2b133595fd4ed2b3994907fa_BDF_BNAS_VLP_TYPE2.

<ipython-input-17-79cb622c246c>:46: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file, sr=None)



E:/TESIS/DataSet/AudioSismig/4d7148e0aae975fe48c0ee610ac285ca_BHZ_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/1fecb2c616ee2869a02c853fde528083_BHN_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/bd73b3338746b508015c39df899f7b4d_BHZ_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/7fcd1b4245118c4c7e2bff681e4f267b_BHZ_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/b24b7e52f50cd3ecb58828853f15340f_BDF_BNAS_VLP_TYPE1.wav

E:/TESIS/DataSet/AudioSismig/f8feb376ecaf69684add132a25b01073_BHZ_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/ed31672125758dac5313e525af133825_BHN_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/865065924c9caf8849cd3b9372588dc9_BHE_BNAS_VLP_TYPE1.wav

E:/TESIS/DataSet/AudioSismig/4aecb0ba2b133595fd4ed2b3994907fa_BHE_BNAS_VLP_TYPE2.wav

E:/TESIS/DataSet/AudioSismig/4de1ce75862558c9a2debd0e7c08e755_BHZ_BTAM_LP.wav

E:/TESIS/DataSet/AudioSismig/ff6667436f3a3a206d1f118076c80e34_BHE_BNAS_VLP_TYPE1.wav

E:/TESIS/DataSet/AudioSismig/f0ea2d4f1b76874a87fde499f3837d75_BHE_BNAS_VLP_TYPE1.wav

E:/TESIS/DataSet

In [18]:
class ANSYN_Dataset_SE(torch.utils.data.Dataset):

    def __init__(self, filenames, audio_path):
        self.filenames = filenames
        self.audio_path = audio_path
    
    def process_audio(self, signal, new_sr):
        try:
            # right pad if necessary
            length_signal = signal.shape[1]
            if length_signal < 53363:
                num_missing_samples = 53363 - length_signal
                last_dim_padding = (0, num_missing_samples)
                signal = torch.nn.functional.pad(signal, last_dim_padding)
            elif length_signal > 53363:
                signal = signal[:, :53363]
            return signal
        except Exception as e:
            # Manejar errores aquí, por ejemplo, imprimir un mensaje de error y devolver un tensor vacío
            print(f"Error al procesar el audio: {str(e)}")
            return torch.zeros((1, 53363))

    def normalize_layer(self, feats):
        with torch.no_grad():
            feats = torch.nn.functional.layer_norm(feats, feats.shape)
        return feats

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        # Intenta cargar el archivo de audio
        feats, _ = torchaudio.load(self.audio_path + self.filenames[index])
            
        # Procesa el audio y normalízalo
        feats = self.process_audio(feats, 16000)
        feats = self.normalize_layer(feats)

        # Asigna una etiqueta de destino basada en el nombre del archivo
        if 'EXPL' in self.filenames[index]:
            target = torch.tensor(int('00')).long() 
        elif 'HB' in self.filenames[index]:
            target = torch.tensor(int('01')).long() 
        elif 'LP' in self.filenames[index]:
            target = torch.tensor(int('02')).long()
        elif 'TRBA' in self.filenames[index]:
            target = torch.tensor(int('03')).long()
        elif 'TRESP' in self.filenames[index]:
            target = torch.tensor(int('04')).long()
        elif 'VLP' in self.filenames[index]:
            target = torch.tensor(int('05')).long()
        elif 'VT' in self.filenames[index]:
            target = torch.tensor(int('06')).long()

        return {"input_values": feats, "target": target}
        

In [20]:
X_train = os.listdir('D:/USFQ/Noveno Semestre/Proyecto Integrador/Project/QuakeWavNet/data/train/')
X_test = os.listdir('D:/USFQ/Noveno Semestre/Proyecto Integrador/Project/QuakeWavNet/data/test/')
X_val = os.listdir('D:/USFQ/Noveno Semestre/Proyecto Integrador/Project/QuakeWavNet/data/validation/')

In [21]:
print(X_train[0])

0002a32d9f7576599e5f41f7333c0098_BDF_BNAS_VLP_TYPE1.wav


In [24]:
train_dataset = ANSYN_Dataset_SE(X_train, 'D:/USFQ/Noveno Semestre/Proyecto Integrador/Project/QuakeWavNet/data/train/')
val_dataset =  ANSYN_Dataset_SE(X_val, 'D:/USFQ/Noveno Semestre/Proyecto Integrador/Project/QuakeWavNet/data/validation/')
test_dataset = ANSYN_Dataset_SE(X_test, 'D:/USFQ/Noveno Semestre/Proyecto Integrador/Project/QuakeWavNet/data/test/')

In [25]:
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"Frequência de amostragem aceita pelo modelo: {target_sampling_rate}")
# Conferindo se os dados de entrada não geram erro no processor
inputs = processor(train_dataset[5]["input_values"], sampling_rate=target_sampling_rate, return_tensors="pt")
print(f'Input values dimensão: {inputs["input_values"].shape}')
print(inputs)

Frequência de amostragem aceita pelo modelo: 16000
Input values dimensão: torch.Size([1, 1, 53363])
{'input_values': tensor([[[-0.7322, -0.7037, -0.6936,  ...,  0.3114,  0.3114,  0.3114]]]), 'attention_mask': tensor([[1]], dtype=torch.int32)}


In [26]:
print('Dimensões de entrada do modelo:')
with torch.no_grad():
    outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

print('Dimensões de saída do modelo: \n',last_hidden_states.shape)

Dimensões de entrada do modelo:
Dimensões de saída do modelo: 
 torch.Size([1, 333, 768])


In [27]:
@dataclass
class DataCollatorWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["target"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        #print('batch', batch)
        with self.processor.as_target_processor(): labels_batch = self.processor.pad( label_features, padding=True,max_length=self.max_length_labels,pad_to_multiple_of=self.pad_to_multiple_of_labels,return_tensors="pt",)
        print('labels_batch', labels_batch)


        labels = labels_batch["input_ids"]

        batch["target"] = labels

        return batch

In [28]:
data_collator = DataCollatorWithPadding(processor=processor,
                                        #max_length=188,
                                        padding=True)

In [29]:
batch_size = hparams["bs"]

train_dataloader = DataLoader(train_dataset, batch_size=batch_size,
                              collate_fn = data_collator,
                              shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size,
                            collate_fn = data_collator,
                            shuffle=False, num_workers=4)

print('Número de minibatches de treinamento:', len(train_dataloader))
print('Número de minibatches de validação:', len(val_dataloader))

batch = next(iter(train_dataloader))
#print(batch)

x_train, y_train = batch['input_values'], batch['target']
print("\nDimensões dos dados de um minibatch - Audio:", x_train.size())
# print("\nDimensões dos dados de um minibatch:", padding_mask.size())
print("\nDimensões dos dados de um minibatch - Target:", y_train.size())
print("Valores mínimo e máximo entrada: ", torch.min(x_train), torch.max(x_train))
print("Valores mínimo e máximo saída: ", torch.min(y_train), torch.max(y_train))
print("Tipo dos dados dos áudios:         ", type(x_train))
print("Tipo das classes das classes:       ", type(y_train))

Número de minibatches de treinamento: 3803
Número de minibatches de validação: 816


In [ ]:
n_classes = 7
f1 = F1Score(num_classes=n_classes, average='macro', task='multiclass')
accuracy = Accuracy(num_classes=n_classes,task='multiclass')

In [ ]:
from sklearn.metrics import f1_score


In [ ]:
class Wav2Vec2_sound_detection(pl.LightningModule):
    def __init__(self, *args, **kwargs):
        super().__init__()

        self.hparams.update(hparams)

        self.freeze_finetune_updates = hparams["freeze_finetune_updates"]

        #self.model = model4c
        self.val_f1_scores = []
        self.test_f1_scores = []
        
        self.model = Wav2Vec2_ChannelModel.from_pretrained(hparams["pretrained"],
                                                 conv_dim = (512, 512, 512, 512, 512, 512),
                                                 conv_stride = (5, 2, 2, 2, 2, 2),
                                                 conv_kernel = (10, 3, 3, 3, 3, 2),
                                                 num_feat_extract_layers = 6,
                                                 apply_spec_augment=hparams["apply_mask"],
                                                 #mask_time_length=hparams["mask_time_length"],
                                                 num_input_channels = 1,
                                                 ignore_mismatched_sizes=True)


        # self.model.feature_extractor._freeze_parameters()

        # freeze base-model
        # for param in self.model.parameters():
        #     param.requires_grad = False

        self.projector = nn.Linear(self.model.config.hidden_size, self.model.config.classifier_proj_size)
        n_classes = 7
        self.final_layer = nn.Linear(self.model.config.classifier_proj_size, n_classes)

    def forward(self, samples):

        ft = self.freeze_finetune_updates <= self.trainer.global_step

        with torch.no_grad() if not ft else contextlib.ExitStack():
            hidden_states = self.model(**samples).last_hidden_state

        padding_mask = self.model._get_feature_vector_attention_mask(hidden_states.shape[1], samples["attention_mask"])

        hidden_states[~padding_mask] = 0.0

        pooled_output = hidden_states.sum(dim=1) / padding_mask.sum(dim=1).view(-1, 1)

        proj_pooled = self.projector(pooled_output)

        preds = self.final_layer(proj_pooled)

        return F.log_softmax(preds, dim=1)

    def _get_feature_vector_attention_mask(self, feature_vector_length: int, attention_mask: torch.LongTensor):
        output_lengths = self._get_feat_extract_output_lengths(attention_mask.sum(-1)).to(torch.long)
        batch_size = attention_mask.shape[0]

        attention_mask = torch.zeros(
            (batch_size, feature_vector_length), dtype=attention_mask.dtype, device=attention_mask.device
        )

        attention_mask[(torch.arange(attention_mask.shape[0], device=attention_mask.device), output_lengths - 1)] = 1
        attention_mask = attention_mask.flip([-1]).cumsum(-1).flip([-1]).bool()
        return attention_mask
        
        
    def training_step(self, train_batch, batch_idx):

        y_value = train_batch.pop("target")
        log_softs = self.forward(train_batch)
    

        loss = F.nll_loss(log_softs, y_value)
        
        
        

        self.log('loss_step', loss, on_step=True, prog_bar=True)
        

        return loss

    def training_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean()
        

        self.log("train_loss", loss, prog_bar=True)
        
        
        

    def validation_step(self, val_batch, batch_idx):

        y_value = val_batch.pop("target")

        log_softs = self.forward(val_batch)
        preds = torch.argmax(log_softs, dim=1)

        val_acc = accuracy(preds.cpu(), y_value.cpu())
        val_f1 = f1(preds.cpu(), y_value.cpu())
        val_loss = F.nll_loss(log_softs, y_value)

        self.log('val_acc', val_acc, prog_bar=True)
        self.log('val_f1', val_f1, prog_bar=True)
        self.log('val_loss', val_loss, prog_bar=True)

        return {"val_acc_step": val_acc, "val_f1_step": val_f1, "val_loss_step": val_loss}

    
    def validation_epoch_end(self, outputs):
        acc_mean = torch.stack([x['val_acc_step'] for x in outputs]).mean()
        f1_mean = torch.stack([x['val_f1_step'] for x in outputs]).mean()
        loss_mean = torch.stack([x['val_loss_step'] for x in outputs]).mean()

        self.log("val_acc", acc_mean, prog_bar=True)
        self.log("val_f1", f1_mean, prog_bar=True)
        self.log("val_loss", loss_mean, prog_bar=True)

        self.val_f1_scores.append(f1_mean)
    import torch.functional as F
    
    """def validation_epoch_end(self, outputs):
        acc_mean = torch.stack([x['val_acc_step'] for x in outputs]).mean()
        f1_mean = torch.stack([x['val_f1_step'] for x in outputs]).mean()
        loss_mean = torch.stack([x['val_loss_step'] for x in outputs]).mean()

        self.log("val_acc", acc_mean, prog_bar=True)
        self.log("val_f1", f1_mean, prog_bar=True)
        self.log("val_loss", loss_mean, prog_bar=True)"""

    def test_step(self, test_batch, batch_idx):

        y_value = test_batch.pop("target")

        log_softs = self.forward(test_batch)
        preds = torch.argmax(log_softs, dim=1)

        test_acc = accuracy(preds.cpu(), y_value.cpu())
        test_f1 = f1(preds.cpu(), y_value.cpu())
        test_loss = F.nll_loss(log_softs, y_value)

        self.log('test_acc', test_acc, prog_bar=True)
        self.log('test_f1', test_f1, prog_bar=True)
        self.log('test_loss', test_loss, prog_bar=True)

        return {"test_acc_step": test_acc, "test_f1_step": test_f1,  "test_loss_step": test_loss}

    def test_epoch_end(self, outputs):
        acc_mean = torch.stack([x['test_acc_step'] for x in outputs]).mean()
        f1_mean = torch.stack([x['test_f1_step'] for x in outputs]).mean()
        loss_mean = torch.stack([x['test_loss_step'] for x in outputs]).mean()

        self.log("test_acc", acc_mean, prog_bar=True)
        self.log("test_f1", f1_mean, prog_bar=True)
        self.log("test_loss", loss_mean, prog_bar=True)
        
        self.test_f1_scores.append(f1_mean)

    def configure_optimizers(self):

        optimizer = torch.optim.Adam(self.parameters(),
                         lr=self.hparams["lr"],
                         betas=(0.9,0.98),
                         eps=1e-6,
                         weight_decay=self.hparams["w_decay"])

        scheduler = LinearWarmupCosineAnnealingLR(optimizer,
                                                  eta_min=0,
                                                  warmup_start_lr=self.hparams["lr"],
                                                  warmup_epochs=self.hparams["warmup_epochs"],
                                                  max_epochs=self.hparams["max_epochs"])

        return {'optimizer': optimizer, 'lr_scheduler': scheduler}

In [ ]:
neptune_logger = NeptuneLogger(
    api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJjMWEyNTJlZS05ZDI5LTQzZjktYTkzNy00MDczMmZhODU3OWUifQ==",
    project='kgrosero/IA025-Project-wav2vec2')

In [ ]:
model = Wav2Vec2_sound_detection(hparams)

trainer = pl.Trainer(gpus=1,
                     logger=neptune_logger,
                     max_epochs=100,
                    overfit_batches=3,
                    log_every_n_steps = 1)

trainer.fit(model, train_dataloader, val_dataloader)
del model, trainer # Para não ter estouro de mémoria da GPU
#gc.collect()
torch.cuda.empty_cache()

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2_ChannelModel: ['lm_head.weight', 'wav2vec2.feature_extractor.conv_layers.6.conv.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2_ChannelModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2_ChannelModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2_ChannelModel were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cud

Sanity Checking: 0it [00:00, ?it/s]

labels_batchlabels_batch  {'input_ids': tensor([2, 2, 6, 2, 6, 6, 6, 2, 2, 2, 2, 2, 6, 6, 2, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}{'input_ids': tensor([1, 6, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 1, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

labels_batch {'input_ids': tensor([6, 2, 6, 2, 2, 6, 2, 6, 6, 2, 2, 2, 6, 6, 6, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
labels_batch {'input_ids': tensor([6, 2, 6, 6, 2, 2, 6, 6, 2, 2, 2, 6, 2, 2, 2, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}labels_batch
 {'input_ids': tensor([2, 2, 6, 6, 2, 1, 2, 2, 6, 2, 6, 2, 6, 4, 2, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}labels_batch
 {'input_ids': tensor([0, 2, 2, 2, 2, 6, 6, 2, 6, 6, 2, 6, 6, 2, 2, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
labels_batch labels_batch{'input_i

Training: 0it [00:00, ?it/s]

labels_batchlabels_batch  {'input_ids': tensor([6, 2, 2, 6, 2, 2, 6, 2, 2, 2, 2, 6, 2, 2, 2, 2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}{'input_ids': tensor([6, 2, 2, 6, 2, 6, 2, 6, 6, 6, 6, 2, 1, 2, 2, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

labels_batch {'input_ids': tensor([2, 2, 2, 2, 6, 2, 6, 6, 6, 2, 2, 1, 2, 2, 2, 2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
labels_batchlabels_batchlabels_batchlabels_batch    {'input_ids': tensor([2, 2, 1, 2, 2, 2, 6, 2, 1, 6, 2, 2, 6, 6, 6, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}{'input_ids': tensor([6, 6, 2, 2, 2, 6, 2, 1, 2, 2, 1, 2, 6, 6, 6, 2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}{'input_ids': tensor([6, 6, 2, 2, 2, 2, 6, 2, 6, 2, 2, 6, 6, 1, 6, 2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
{'input_ids': tensor([6

...

Validation: 0it [00:00, ?it/s]

labels_batchlabels_batch  {'input_ids': tensor([1, 6, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 1, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}{'input_ids': tensor([6, 2, 6, 2, 2, 6, 2, 6, 6, 2, 2, 2, 6, 6, 6, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
labels_batch
 labels_batch {'input_ids': tensor([2, 2, 6, 2, 6, 6, 6, 2, 2, 2, 2, 2, 6, 6, 2, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}{'input_ids': tensor([6, 2, 6, 6, 2, 2, 6, 6, 2, 2, 2, 6, 2, 2, 2, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

labels_batch labels_batchlabels_batch{'input_ids': tensor([0, 2, 2, 2, 2, 6, 6, 2, 6, 6, 2, 6, 6, 2, 2, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}  
{'input_ids': tensor([6, 2, 6, 2, 2, 2, 2, 2, 2, 2, 6, 6, 6, 2, 2, 2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}{'input_ids': tensor([

Validation: 0it [00:00, ?it/s]

labels_batch {'input_ids': tensor([1, 6, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 1, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
labels_batchlabels_batchlabels_batch  labels_batch  {'input_ids': tensor([0, 2, 2, 2, 2, 6, 6, 2, 6, 6, 2, 6, 6, 2, 2, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}{'input_ids': tensor([6, 2, 6, 2, 2, 6, 2, 6, 6, 2, 2, 2, 6, 6, 6, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

{'input_ids': tensor([2, 2, 6, 2, 6, 6, 6, 2, 2, 2, 2, 2, 6, 6, 2, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}{'input_ids': tensor([6, 2, 6, 6, 2, 2, 6, 6, 2, 2, 2, 6, 2, 2, 2, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

labels_batch labels_batch{'input_ids': tensor([6, 2, 6, 2, 2, 2, 2, 2, 2, 2, 6, 6, 6, 2, 2, 2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])} 
{'input_ids': tensor(

`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
pl_model= Wav2Vec2_sound_detection(hparams=hparams)
checkpoint_path = '/home/cslab03/Desktop/QuakeWavNet/Results/'
checkpoint_dir = os.path.dirname(os.path.abspath(checkpoint_path))
print(f'Files in {checkpoint_path}: {os.listdir(checkpoint_path)}')
print(f'Saving checkpoints to {checkpoint_path}')
checkpoint_callback = pl.callbacks.ModelCheckpoint(filename=hparams["version"],
                                                  dirpath=checkpoint_dir,
                                                  save_top_k=1,
                                                  verbose = True,
                                                  monitor="val_f1", mode="max")
early_stop_callback = pl.callbacks.EarlyStopping(monitor="val_f1", patience=hparams["patience"], mode='max')
lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='step')

trainer = pl.Trainer(gpus=1,
                     precision=16,
                     logger=neptune_logger,
                     num_sanity_val_steps=0,
                     accumulate_grad_batches=hparams["accum_grads"],
                     enable_checkpointing=True,
                     callbacks=[early_stop_callback, lr_monitor, checkpoint_callback],
                     max_epochs=50)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2_ChannelModel: ['lm_head.weight', 'wav2vec2.feature_extractor.conv_layers.6.conv.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2_ChannelModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2_ChannelModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2_ChannelModel were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using 16bit None Automat

Files in /home/cslab03/Desktop/QuakeWavNet/Results: []
Saving checkpoints to /home/cslab03/Desktop/QuakeWavNet/Results


In [ ]:
# Definir listas para almacenar las métricas por época
train_losses = []
val_losses = []
val_f1_scores = []

torch.cuda.empty_cache()
trainer.fit(pl_model, train_dataloader, val_dataloader)

You are using a CUDA device ('NVIDIA RTX A4000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/tmp/ipykernel_6728/949965054.py:166: UnderReviewWarning: The feature LinearWarmupCosineAnnealingLR is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  scheduler = LinearWarmupCosineAnnealingLR(optimizer,

  | Name        | Type                  | Params
------------------------------------------------------
0 | model       | Wav2Vec2_ChannelModel | 93.8 M
1 | proje

Training: 0it [00:00, ?it/s]

labels_batch {'input_ids': tensor([6, 2, 4, 2, 6, 6, 6, 6, 6, 2, 6, 2, 6, 6, 1, 2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
labels_batchlabels_batch  {'input_ids': tensor([2, 2, 6, 2, 2, 2, 1, 2, 6, 2, 2, 2, 2, 2, 2, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}{'input_ids': tensor([2, 6, 2, 6, 6, 6, 2, 1, 6, 2, 2, 6, 2, 6, 2, 2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

labels_batch labels_batch {'input_ids': tensor([2, 6, 2, 2, 6, 6, 6, 6, 6, 2, 6, 2, 6, 2, 2, 2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
{'input_ids': tensor([2, 2, 6, 1, 2, 6, 2, 1, 6, 6, 6, 6, 2, 2, 6, 2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
labels_batch labels_batch{'input_ids': tensor([6, 2, 6, 2, 2, 6, 2, 2, 2, 2, 2, 6, 2, 2, 6, 6]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])} 
{'input_ids': tensor(

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/cslab03/anaconda3/envs/coEnv/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/cslab03/anaconda3/envs/coEnv/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/cslab03/anaconda3/envs/coEnv/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/ipykernel_6728/419823214.py", line 28, in __getitem__
    feats, _ = torchaudio.load(self.audio_path + self.filenames[index])
  File "/home/cslab03/anaconda3/envs/coEnv/lib/python3.9/site-packages/torchaudio/backend/sox_io_backend.py", line 256, in load
    return _fallback_load(filepath, frame_offset, num_frames, normalize, ch

In [ ]:
if not os.path.isfile(checkpoint_path + hparams["version"]):
    print('Saving processor to: ' + checkpoint_path + hparams["version"])
    processor.save_pretrained(checkpoint_path + hparams["version"])

In [ ]:
checkpoint_callback.best_model_path
import matplotlib.pyplot as plt

# Después de completar el entrenamiento
epochs = list(range(1, len(pl_model.val_f1_scores) + 1))
plt.plot(epochs, pl_model.val_f1_scores, marker='o')
plt.xlabel('Época')
plt.ylabel('F1-score')
plt.title('F1-score por Época')
plt.grid(True)
plt.show()

In [ ]:
best_model = checkpoint_callback.best_model_path
# best_model = "/content/drive/MyDrive/Wav2Vec2_ORVP/wav2vec2_huggingface_fairseq_orvp_test1-epoch=4-step=23459.ckpt"
print(best_model)
test_model = Wav2Vec2_sound_detection.load_from_checkpoint(best_model, hparams=hparams).cuda().eval()

In [ ]:
trainer.test(test_model, val_dataloader)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size,
                            collate_fn = data_collator,
                            shuffle=False, num_workers=0)

In [ ]:
trainer.test(test_model, test_dataloader)